In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

In [2]:
import pandas as pd
import psycopg2
from decouple import config

# connect to database
conn = psycopg2.connect(
    host=config('POSTGRES_HOST'),
    port="5432",
    database="db13f2023",
    user=config('POSTGRES_USER'),
    password=config('POSTGRES_PASSWORD')
)

conn.commit()
cur = conn.cursor()

cur.execute("""
    SELECT
        ticker,
    	sum(value) AS values,
    	sum(shares) AS shares,
        holdings_2021.name_of_issuer,
        filings_2021.period_of_report
    FROM
    	holdings_2021
    JOIN filings_2021
    ON
    	filings_2021.filing_id = holdings_2021.filing_id
    JOIN holding_infos
    ON
    	holdings_2021.cusip = holding_infos.cusip
    WHERE
    	 cik = '1067983'
    GROUP BY security_name, holdings_2021.name_of_issuer, ticker, filings_2021.period_of_report
    ORDER BY VALUES DESC;
""")

rows = cur.fetchall()

cur.close()

buffet_holdings = pd.DataFrame(rows, columns=[
    'ticker',
    'values',
	'shares', 
    'name_of_issuer',
    'period_of_report'])



In [3]:
buffet_holdings.head(10)

,ticker,values,shares,name_of_issuer,period_of_report
0,AAPL,121502087000,887135554,APPLE INC,2021-06-30
1,BAC,41646448000,1010100606,BANK AMER CORP,2021-06-30
2,AXP,25050635000,151610700,AMERICAN EXPRESS CO,2021-06-30
3,KO,21644001000,400000000,COCA COLA CO,2021-06-30
4,KHC,13279388000,325634818,KRAFT HEINZ CO,2021-06-30
5,DUT,8939587000,24669778,MOODYS CORP,2021-06-30
6,BAC,8898941000,158824575,VERIZON COMMUNICATIONS INC,2021-06-30
7,USB,7342802000,128888906,US BANCORP DEL,2021-06-30
8,TRL,4346990000,36095570,DAVITA INC,2021-06-30
9,CHTR,3761251000,5213461,CHARTER COMMUNICATIONS INC N,2021-06-30
